In [ ]:

import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_json('/kaggle/input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset_v2.json',lines=True)

In [ ]:
df.head()

In [ ]:
df = df.drop(columns=['article_link'])
X = df['headline']
y = df['is_sarcastic']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=101)

In [ ]:
embedding = "https://tfhub.dev/google/universal-sentence-encoder/4"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode='min',patience=0,verbose=1,)

In [ ]:
history = model.fit(x_train,y_train,batch_size=256,
                    epochs=10,
                    validation_data=(x_test,y_test),
                    verbose=1,callbacks=[earlystop])

In [ ]:
model_eval = model.evaluate(x_test,y_test)
print('Validation_loss: '+str(model_eval[0]))
print('Accuracy: '+str(model_eval[1]))